In [2]:
using Pkg
Pkg.activate(@__DIR__);
Pkg.instantiate()

In [3]:
using CSV
using DataFrames
using Plots; plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /Users/AlexanderBouman/.julia/packages/Plots/qbc7U/src/backends.jl:432


Plots.PlotlyBackend()

In [1]:
df1 = CSV.read("processed_data/merged_2021-02-03-13-43-38_seg_1.csv", DataFrame)
df2 = CSV.read("processed_data/merged_2021-02-03-13-43-38_seg_2.csv", DataFrame)
df3 = CSV.read("processed_data/merged_2021-02-03-13-43-38_seg_3.csv", DataFrame)
df4 = CSV.read("processed_data/merged_2021-02-03-13-43-38_seg_4.csv", DataFrame)
names(df4)

# convert(Matrix, ExampleInputDF[!, ["pos x","pos y","pos z"]])
# scatter(Matrix(ExampleInputDF[!, ["pos x","pos y","pos z"]]))
# scatter(ExampleInputDF[!, "pos x"],ExampleInputDF[!, "pos y"],ExampleInputDF[!, "pos z"])

UndefVarError: UndefVarError: CSV not defined

In [ ]:
plt = plot3d(
    1,
    xlim = (-30, 30),
    ylim = (-30, 30),
    zlim = (0, 60),
    title = "Lorenz Attractor",
    marker = 2,
)

# build an animated gif by pushing new points to the plot, saving every 10th frame
@gif for i=1:1500
    step!(attractor)
    push!(plt, attractor.x, attractor.y, attractor.z)
end every 10

In [47]:
?plot3d!

search: plot3d! plot3d



No documentation found.

`Plots.plot3d!` is a `Function`.

```
# 1 method for generic function "plot3d!":
[1] plot3d!(args...; kw...) in Plots at /Users/AlexanderBouman/.julia/packages/Plots/qbc7U/src/shorthands.jl:410
```


In [56]:
ExampleInputDF